In [2]:
# import packages
import numpy as np
import matplotlib.pyplot as plt
import pyphysio as ph

In [3]:
%matplotlib auto

Using matplotlib backend: Qt5Agg


In [4]:
ecg = ph.from_pickle('/home/bizzego/UniTn/data/nicola/pkl/ecg/L21_M.pkl')
eda = ph.from_pickle('/home/bizzego/UniTn/data/nicola/pkl/eda/L21_M.pkl')
emg = ph.from_pickle('/home/bizzego/UniTn/data/nicola/pkl/emg/L21_M.pkl')

In [5]:
# plot
ax1 = plt.subplot(311)
ecg.plot()
plt.subplot(312, sharex=ax1)
eda.plot()
plt.subplot(313, sharex=ax1)
emg.plot()

In [ ]:
# resampling
ecg_128 = ecg.resample(fout=128)

plt.figure()
ecg.plot() # plotting the original signal
ecg_128.plot('or') # plotting the samples of the downsampled signal
plt.xlim((40,42)) # setting the range of the x axis between 40 and 42 seconds

In [ ]:
f, psd = ph.PSD(method='welch')(ecg)

In [ ]:
plt.figure()
plt.plot(f, psd)
ax = plt.gca()
ax.set_yscale('log')

In [ ]:
lowpass_filter = ph.IIRFilter(fp=25, fs=30, ftype='ellip')

In [ ]:
ecg_lp = lowpass_filter(ecg)

f, psd_lp = ph.PSD(method='welch')(ecg_lp)
plt.plot(f, psd_lp)

In [ ]:
plt.figure()
ecg.plot()
ecg_lp.plot()

In [ ]:
highpass_filter = ph.IIRFilter(fp=5, fs=2, ftype='ellip')

In [ ]:
ecg_hp = highpass_filter(ecg_lp)

f, psd_hp = ph.PSD(method='fft')(ecg_hp)

plt.figure()
plt.plot(f, psd)
plt.plot(f, psd_lp)
plt.plot(f, psd_hp)
ax = plt.gca()
ax.set_yscale('log')

In [ ]:
ecg.plot()
ecg_lp.plot()
ecg_hp.plot()

In [ ]:
ibi = ph.BeatFromECG()(ecg_hp)

In [ ]:
plt.figure()
ax1 = plt.subplot(211)
ecg_hp.plot()
plt.vlines(ibi.get_times(), ecg_hp.min(), ecg_hp.max(), 'red')
plt.subplot(212, sharex=ax1)
ibi.plot('o')
plt.vlines(ibi.get_times(), ibi.min(), ibi.max(), 'red')



In [ ]:
fixed_length = ph.FixedSegments(step = 30, width = 60)

results, names = ph.fmap(fixed_length, ph.preset_hrv_fd(), ibi.resample(4))

In [ ]:
print(np.c_[np.arange(len(names)), names])

In [ ]:
plt.plot(results[:,5]) #HF indicator, associated to parasympathetic